In [ ]:
import pandas as pd

data = pd.read_csv('news.csv', index_col=0)
data.head()

In [ ]:
data_copy = data.copy()
data_copy["Sent_label"] = data_copy["Sentiment"].map(lambda x: "positive" if x > 0.05 else "negative" if x < -0.05 else "neutral")
data_copy["Org_label"] = data_copy["Org"].map(lambda x: x if len(x) > 0 else "No orgs" )

targets = ["headline", "Date scraped", "Topics", "Sent_label", "Org_label"]
data_copy["metadatas"] = data_copy[targets].apply(lambda x: x.to_dict(), axis=1)

In [ ]:
data_copy["metadatas"]

In [ ]:
import os
os.environ["ANONYMIZED_TELEMETRY"] = "false"

In [ ]:
import chromadb
from chromadb.utils import embedding_functions
server = chromadb.PersistentClient(path="v_db1")
embedder =  embedding_functions.SentenceTransformerEmbeddingFunction(model_name="paraphrase-multilingual-MiniLM-L12-v2", normalize_embeddings=True)

In [ ]:
collection =  server.get_or_create_collection(
                                            "articles",
                                            embedding_function=embedder,
                                            metadata={
                                                "title": "News article",
                                                "description": "to provide",
                                                "author": "mthiaw",
                                            }
                                            )

In [ ]:
collection.add(
    ids = data_copy["Unique ID"].tolist(),
    documents = data_copy["body"].tolist(),
    metadatas= data_copy["metadatas"].tolist()
)

In [ ]:
res = collection.query(
    query_texts="guerre entre israel et palestine",
    n_results=5,

)
res

In [ ]:
texte = "Ceci est un exemple de texte très long à formater proprement."
print(f"{texte[:125]}...")

In [ ]:
for i in range(5):
    print('-', res["metadatas"][0][i])